In [1]:
/* 21-6 */
USE tempdb;
GO

Commands completed successfully.

Total execution time: 00:00:00.002

In [2]:
CREATE TABLE Creditor(
CreditorID		INT IDENTITY PRIMARY KEY,
CreditorName	VARCHAR(50)
);
GO

Commands completed successfully.

Total execution time: 00:00:00.011

In [3]:
INSERT INTO Creditor
VALUES('You Owe Me'), 
	('You Owe Me More');
GO

(2 rows affected)

Total execution time: 00:00:00.006

In [4]:
SELECT *
FROM Creditor;
GO

(2 rows affected)

Total execution time: 00:00:00.020

CreditorID,CreditorName
1,You Owe Me
2,You Owe Me More


In [5]:
/* Create a trigger and test */
USE tempdb;
GO

CREATE TRIGGER Deny_Delete
ON Creditor
FOR DELETE
AS
RAISERROR('Deletions are not permitted', 16, 1)
ROLLBACK TRANSACTION;
GO

Commands completed successfully.

: Msg 8197, Level 16, State 4, Procedure Deny_Delete, Line 2
The object 'Creditor' does not exist or is invalid for this operation.

Total execution time: 00:00:00.011

In [6]:
DELETE 
FROM Creditor
WHERE CreditorID = 1;
GO

(1 row affected)

Total execution time: 00:00:00.007

In [7]:
/* solution 2 */
THROW 50000, 'User defined error', 1;

: Msg 50000, Level 16, State 1, Line 2
User defined error

Total execution time: 00:00:00.007

In [8]:
/* improperly declared raiserror */

USE tempdb;
GO

BEGIN TRY
  PRINT 'Outer Try'
    BEGIN TRY
	 	PRINT ERROR_NUMBER() + ' Inner try'
    END TRY
    BEGIN CATCH
        IF ERROR_NUMBER() = 8134
		   PRINT CONVERT(CHAR(5), ERROR_NUMBER()) + ' Inner Catch Divide by zero'
        ELSE 
            BEGIN
            PRINT CONVERT(CHAR(6), ERROR_NUMBER()) + ' '
			 + ERROR_MESSAGE() +
              CONVERT(CHAR(2), ERROR_SEVERITY()) + ' ' +
              CONVERT(CHAR(2), ERROR_STATE()) + ' INITIAL Catch';
           RAISERROR --This THROW is added in the initial CATCH
           END
    END CATCH;
END TRY
BEGIN CATCH
    IF ERROR_NUMBER() = 8134
        PRINT CONVERT(CHAR(5), ERROR_NUMBER()) + ' Outer Catch Divide by zero' 
    ELSE 
        BEGIN
        PRINT CONVERT(CHAR(6), ERROR_NUMBER()) + ' ' + ERROR_MESSAGE() +
              CONVERT(CHAR(2), ERROR_SEVERITY()) + ' ' + 
              CONVERT(CHAR(2), ERROR_STATE()) + ' OUTER Catch';
        RAISERROR
        END
END CATCH;

Commands completed successfully.

: Msg 156, Level 15, State 1, Line 21
Incorrect syntax near the keyword 'END'.

: Msg 156, Level 15, State 1, Line 33
Incorrect syntax near the keyword 'END'.

Total execution time: 00:00:00.008

In [9]:
/* properly defined RAISERROR */
USE tempdb;
GO

BEGIN TRY
  PRINT 'Outer Try'
    BEGIN TRY
	 	PRINT ERROR_NUMBER() + ' Inner try'
    END TRY
    BEGIN CATCH
        DECLARE @error_message AS VARCHAR(500) = ERROR_MESSAGE()
        DECLARE @error_severity AS INT = ERROR_SEVERITY()
        DECLARE @error_state AS INT = ERROR_STATE()

        IF ERROR_NUMBER() = 8134
		   PRINT CONVERT(CHAR(5), ERROR_NUMBER()) + ' Inner Catch Divide by zero'
        ELSE 
            BEGIN
            PRINT CONVERT(CHAR(6), ERROR_NUMBER()) + ' '
			 + ERROR_MESSAGE() +
              CONVERT(CHAR(2), ERROR_SEVERITY()) + ' ' +
              CONVERT(CHAR(2), ERROR_STATE()) + ' INITIAL Catch';
           RAISERROR(@error_message,@error_severity,@error_state); 
           END
    END CATCH;
END TRY
BEGIN CATCH
    IF ERROR_NUMBER() = 8134
        PRINT CONVERT(CHAR(5), ERROR_NUMBER()) + ' Outer Catch Divide by zero' 
    ELSE 
        BEGIN
        PRINT CONVERT(CHAR(6), ERROR_NUMBER()) + ' ' + ERROR_MESSAGE() +
              CONVERT(CHAR(2), ERROR_SEVERITY()) + ' ' + 
              CONVERT(CHAR(2), ERROR_STATE()) + ' OUTER Catch';
        RAISERROR(@error_message,@error_severity,@error_state);

        END
END CATCH;

Commands completed successfully.

Outer Try

245 Conversion failed when converting the varchar value ' Inner try' to data type int.16 1 INITIAL Catch

50000 Conversion failed when converting the varchar value ' Inner try' to data type int.16 1 OUTER Catch

: Msg 50000, Level 16, State 1, Line 35
Conversion failed when converting the varchar value ' Inner try' to data type int.

Total execution time: 00:00:00.014